# GRASS setup in Windows 
 - Note: Setup with QGIS does not work because the grass executable does not exist. A standalone installation of GRASS is required. 

## Step 1: Test and install six module if not installed
- pip install six

## Step 2: Add grass path to the site-packages
- Go to your python site-packages folder and add a new file named grass.pth
- Then write these two lines in the file
```
C:\Program Files\GRASS GIS 7.8\etc\python
C:\Program Files\GRASS GIS 7.8\bin
```

- Now you can import grass in your python script

## Step 3: If GRASS still not recognised by IDE. go to environment variables in Windows and add new system variables
- GISBASE C:\Program Files\GRASS GIS 7.8
- GISRC E:\AD4GD\Water\grassdata (this is your grassdata folder)
- Path C:\Program Files\GRASS GIS 7.8\bin (edit path and add this)



In [ ]:
import os

# get environment variables
matches_directory = './matches.txt'
output_directory = os.environ['GRASS_OUTPUT_DIRECTORY']
gisdb = os.environ['GRASS_GISDB']
location = os.environ['GRASS_LOCATION_NAME']
mapset = os.environ['GRASS_MAPSET_NAME']
grass7bin_win = os.environ['GRASS_WIN_EXECUTABLE']

In [ ]:
# reference: https://grasswiki.osgeo.org/wiki/Working_with_GRASS_without_starting_it_explicitly
import os
import sys
import subprocess

########### SOFTWARE
# we assume that the GRASS GIS start script is available and in the PATH
# query GRASS 7 itself for its GISBASE
grass7bin = grass7bin_win

# query GRASS 7 itself for its GISBASE
startcmd = [grass7bin, '--config', 'path']

p = subprocess.Popen(startcmd, shell=False,
                     stdout=subprocess.PIPE, stderr=subprocess.PIPE)
out, err = p.communicate()
if p.returncode != 0:
    print >>sys.stderr, "ERROR: Cannot find GRASS GIS 7 start script (%s)" % startcmd
    sys.exit(-1)
gisbase = out.decode('utf-8').strip('\n\r')

# Set GISBASE environment variable
os.environ['GISBASE'] = gisbase
# the following not needed with trunk
os.environ['PATH'] += os.pathsep + os.path.join(gisbase, 'extrabin')
# add path to GRASS addons
home = os.path.expanduser("~")
os.environ['PATH'] += os.pathsep + os.path.join(home, '.grass7', 'addons', 'scripts')

# define GRASS-Python environment
gpydir = os.path.join(gisbase, "etc", "python")
sys.path.append(gpydir)

# Set GISDBASE environment variable
os.environ['GISDBASE'] = gisdb


# Start GRASS Session

In [ ]:
# Set GISDBASE environment variable
os.environ['GISDBASE'] = gisdb
 
# import GRASS Python bindings (see also pygrass)
import grass.script as gscript
import grass.script.setup as gsetup
 
###########
# launch session
gsetup.init(gisbase,
            gisdb, location, mapset)
 
# check is everything is alright (should return gisdb, location and mapset)
print(gscript.gisenv())

# Single thread mode (502 mins)

In [ ]:
class SingleTileProcessor():
    def __init__(self, matches_file_path, output_dir):
        self.matches = self.read_matches_file_to_dict(matches_file_path)
        self.output_dir = output_dir

    def read_matches_file_to_dict(self,file_path):
        matches = {}
        with open(file_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line:
                    key, value = line.split()
                    matches[key] = value
        return matches

    def process_tiles(self, matches:dict):
        # iterate over the matches dictionary
        for key, value in matches.items():
            filename = output_directory + '\\' + key.split('\\')[-1].split('.')[0] + '.csv'
            # import the raster (key) into the mapset
            gscript.run_command('r.in.gdal', input=key, output='py_water')
            # import the raster (value) into the mapset
            gscript.run_command('r.in.gdal', input=value, output='py_cep')
            # set the region to the raster
            gscript.run_command('g.region', raster='py_water')
            # run r.stats to get the statistics (flags: a = print area, n = no nulls)
            gscript.run_command('r.stats', flags='an', input='py_water,py_cep', separator='comma', output=filename)
            # delete the imported rasters
            gscript.run_command('g.remove', flags='f', type='raster', name='py_water,py_cep')

def main():
    tileProcessor = SingleTileProcessor(matches_directory, output_directory)
    tileProcessor.process_tiles()

if __name__ == '__main__':
    main()

# Mutli-Threading: 8 threads (XX mins)
- Note multi-threading may not work in the IDE so run the script in a GRASS session instead

In [ ]:
import multiprocessing

In [ ]:
### ONLY UNCOMMENT WHEN RUNNING IN GRASS ENVIRONMENT ###
##!/usr/bin/env python3

# import grass.script as gscript
# import multiprocessing

# # define variables using absolute paths
# matches_file_path = 'C:\\Users\\riyad\\Documents\\GitHub\\AD4GD_Water\\matches.txt'
# output_dir = 'C:\\Users\\riyad\\Downloads\\Work\\output'
### ONLY UNCOMMENT WHEN RUNNING IN GRASS ENVIRONMENT ###

class MultiTileProcessor:

    def __init__(self, matches_file_path, output_dir, threads=2):
        self.matches = self.read_matches_file_to_dict(matches_file_path)
        self.output_dir = output_dir
        self.threads = threads
        
    def read_matches_file_to_dict(self,file_path):
        matches = {}
        with open(file_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line:
                    key, value = line.split()
                    matches[key] = value
        return matches

    def import_raster(self, match):
        key,value = match
        pid = multiprocessing.current_process().pid
        water_raster = 'py_water_{}'.format(pid)
        cep_raster = 'py_cep_{}'.format(pid)
        combined_raster_names = water_raster + ',' + cep_raster
        # remove the combined raster if it already exists
        gscript.run_command('g.remove', flags='f', type='raster', name=combined_raster_names)
        # import the raster (key) into the mapset
        gscript.run_command('r.in.gdal', input=key, output=water_raster)
        print('Imported: {}'.format(key))
        # import the raster (value) into the mapset
        gscript.run_command('r.in.gdal', input=value, output=cep_raster)
        print('Imported: {}'.format(value))
        filename = key.split('\\')[-1].split('.')[0] + '.csv'
        return combined_raster_names,water_raster,filename
    
    def process_rasters(self,outputs):
        for output in outputs:
            for combined_raster_names,water_raster,filename in output:
                filename = output_directory + '\\' + filename
                #set region to the raster
                gscript.run_command('g.region', raster=water_raster)
                print('Region set to: {}'.format(water_raster))
                # run r.stats to get the statistics (flags: a = print area, n = no nulls)
                gscript.run_command('r.stats', flags='an', input=combined_raster_names, separator='comma', output=filename)
                print('Statistics written to: {}'.format(filename))
                # delete the imported rasters
                gscript.run_command('g.remove', flags='f', type='raster', name=combined_raster_names)
                print('Removed: {}'.format(combined_raster_names))

    
    def process_tiles(self):
        #split matches into chunks depending on the number of threads (4-8 threads is optimal for most systems)
        chunks = [dict(list(self.matches.items())[i:i + self.threads]) for i in range(0, len(self.matches), self.threads)]
        # test with 8 chunks (8 threads)
        chunks = chunks[:8]
        for chunk in chunks:
            print('Processing Chunk: {}'.format(chunk))
            outputs = []
            #import rasters in parallel
            with multiprocessing.Pool(self.threads) as pool:
                outputs.append(pool.map(self.import_raster, chunk.items()))

            #process rasters in on main thread to avoid conflicts with GRASS region settings
            self.process_rasters(outputs)                


def main():
    tileProcessor = MultiTileProcessor(matches_directory, output_directory, threads=8)
    tileProcessor.process_tiles()

if __name__ == '__main__':
    main()